In [74]:
import numpy as np
import copy as copy
import matplotlib.pyplot as plt
from scipy.linalg import norm
Nx=10
Ny=10
L=0.3
H=0.4
N=Nx*Ny
dx=np.zeros(Nx+1)#N
dy=np.zeros(Ny+1)
dx[0][0]=L/Nx
dy[0][0]=L/Ny
for i in range(1,Nx+1):
    for j in range(0,Nx):
        dx[i][j]=dx[i-1][j]*1.1
for i in range(1,Ny+1):
    dy[i]=dy[i-1]*1.1
cx=np.zeros(Nx+1)
cy=np.zeros(Ny+1)
node_x=np.zeros(Nx)
node_y=np.zeros(Ny)
for i in range(1,Nx+1):
    cx[i]=cx[i-1]+dx[i]
for i in range(0,Nx):
    node_x[i]=(cx[i]+cx[i+1])/2
for i in range(1,Ny+1):
    cy[i]=cy[i-1]+dy[i]
for i in range(0,Ny):
    node_y[i]=(cy[i]+cy[i+1])/2 
# [X,Y] =np.meshgrid(x,y)
for i in range(0,Nx+1):
    cx[i]=(cx[i]/cx[-1])*L
for i in range(0,Nx+1):
    cy[i]=(cy[i]/cy[-1])*H

[M,S]=np.meshgrid(cx,cy)
plt.plot(M,cy)
plt.plot(cx,np.transpose(S))
plt.show()
# ..............................cx is storing face value cx[i] and node_x is giving cell center value ......

#..............................................
k= 1000
q=0
qw=500000
T3=100
#Area=1
key=0
epc=10**(-5)
T_0=np.zeros(N,dtype=np.float32)
V=np.zeros(N,dtype=np.float32)
T=np.zeros(N,dtype=np.float32)
residu=[]
itaration=[]
count=0
while(key==0):
    for i in range(0,N):
        if(i==0): #1
            Aw=0
            Ae=k*dy[i]/dx[i]*0.01
            An=k*dx[i]/dy[i]*0.01
            As=0
            Sp=0                            #-(2*k*dy[i])/dx[i] -(2*k*dx[i])/dy[i]
            Su=qw*dy[i]*0.01                 #(q*dy[i]*dx[i])+((2*k*dy[i])/dx[i])*T4+((2*k*dx[i])/dy[i])*T1
            Ap=Aw+Ae+An+As-Sp
            T[i]=(Ae*T[i+1]+An*T[Nx]+Su)/Ap
        elif(i==(Nx -1)):#3
            Aw=(k*dy[i])/dx[i]*0.01
            Ae=0
            An=(k*dx[i])/dy[i]*0.01
            As=0
            Sp=0                          #-(2*k*dy[i])/dx[i]-(2*k*dx[i])/dy[i]
            Su=0                          #(q*dy[i]*dx[i]) + ((2*k*dy[i])/dx[i])*T2 +((2*k*dx[i])/dy[i])*T1
            Ap=Aw+Ae+An+As-Sp
            T[i]=(Aw*T[i-1]+An*T[i+Nx]+Su)/Ap
        elif(i==(N-1)):#9
            Aw=(k*dy[i])/dx[i]*0.01
            Ae=0
            An=0
            As=(k*dx[i])/dy[i]*0.01
            Sp-(2*k*dx[i])/dy[i]*0.01
            Su=+((2*k*dx[i])/dy[i])*T3*0.01
            Ap=Aw+Ae+An+As-Sp
            T[i]=(Aw*T[i-1]+As*T[i-Nx]+Su)/Ap
        elif(i==(N-Nx)):#7done
            Aw=0
            Ae=(k*dy[i])/dx[i]*0.01
            An=0
            As=(k*dx[i])/dy[i]*0.01
            Sp= -(2*k*dx[i])/dy[i]*0.01
            Su=qw*dy[i]*0.01 +((2*k*dx[i])/dy[i])*T3*0.01
            Ap=Aw+Ae+An+As-Sp
            T[i]=(Ae*T[i+1]+As*T[i-Nx]+Su)/Ap
        elif(1<=i<=(Nx-2)):   #2done
            Aw=(k*dy[i])/dx[i]*0.01
            Ae=(k*dy[i])/dx[i]*0.01
            An=(k*dx[i])/dy[i]*0.01
            As=0
            Sp=0
            Su=0
            Ap=Aw+Ae+An+As-Sp
            T[i]=(Ae*T[i+1]+Aw*T[i-1]+An*T[i+Nx]+Su)/Ap
        elif(i%(Nx)==(Nx-1)):#6done 
            Aw=(k*dy[i])/dx[i]*0.01
            Ae=0
            An=(k*dx[i])/dy[i]*0.01
            As=(k*dx[i])/dy[i]*0.01
            Sp=0
            Su=0
            Ap=Aw+Ae+An+As-Sp
            T[i]=(An*T[i+Nx]+As*T[i-Nx]+Aw*T[i-1]+Su)/Ap
        elif((N-Nx)<i<(N-1)):#8
            Aw=(k*dy[i])/dx[i]*0.01
            Ae=(k*dy[i])/dx[i]*0.01
            An=0
            As=(k*dx[i])/dy[i]*0.01
            Sp=-(2*k*dx[i])/dy[i]*0.01
            Su=((2*k*dx[i])/dy[i])*T3*0.01
            Ap=Aw+Ae+An+As-Sp
            T[i]=(Ae*T[i+1]+Aw*T[i-1]+As*T[i-Nx]+Su)/Ap
        elif(i%(Nx-1)==1):#4
            Aw=0
            Ae=(k*dy[i])/dx[i]*0.01
            An=(k*dx[i])/dy[i]*0.01
            As=(k*dx[i])/dy[i]*0.01
            Sp=0
            Su=qw*dy[i]*0.01
            Ap=Aw+Ae+An+As-Sp
            T[i]=(Ae*T[i+1]+As*T[i-Nx]+An*T[i+Nx]+Su)/Ap
        else:#5
            Aw=(k*dy[i])/dx[i]*0.01
            Ae=(k*dy[i])/dx[i]*0.01
            An=(k*dx[i])/dy[i]*0.01
            As=(k*dx[i])/dy[i]*0.01
            Sp=0
            Su=0
            Ap=Aw+Ae+An+As-Sp
            T[i]=(Ae*T[i+1]+Aw*T[i-1]+An*T[i+Nx]+As*T[i-Nx]+Su)/Ap
    #Calculate Resu:
    #print(T)
    #print(T_0)
    count=count+1
    itaration.append(count)            #itaration[j]=count
    #R[j]=(norm(T-T_0))/len(T)
    R=(norm(T-T_0))/len(T)
    residu.append(R)
    if(R<epc):
        key=1
    T_0=copy.copy(T)
    
#print(residu)
T_final=np.reshape(T, (Ny,Nx))
#print(itaration)
[X,Y]=np.meshgrid(node_x,node_y)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(X, Y, T_final)
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Temp Plot')
ax.set_xlabel('x (cm)')
ax.set_ylabel('y (cm)')
plt.savefig("output2.png",dpi=600)
plt.show()
plt.plot(itaration,residu)

TypeError: 'numpy.float64' object does not support item assignment

# mesh written in 1D

In [75]:
import numpy as np
import copy as copy
import matplotlib.pyplot as plt
from scipy.linalg import norm
Nx=5
Ny=5
L=1
H=0.5
N=Nx*Ny
dx_1=np.zeros((Nx,Ny),dtype=np.float32)#N
dy_1=np.zeros((Nx,Ny),dtype=np.float32)
dy_1[0]=H/Ny
for j in range(0,Nx):
    dx_1[j][0]=L/Nx
    for i in range(1,Nx):
            dx_1[j][i]=dx_1[j][i-1]*1.1
for j in range(0,Ny):
    dy_1[0][0]=H/Ny
    for i in range(1,Ny):
        dy_1[i][j]=dy_1[i-1][j]*1.1
dx=dx_1.reshape(-1)
dy=dy_1.reshape(-1)
# print(dy)
cx=np.zeros(Nx+1)
cy=np.zeros(Ny+1)
node_x=np.zeros(Nx+1)
node_y=np.zeros(Ny+1)
deltax_1=np.zeros(Nx)
deltay_1=np.zeros(Ny)
deltax=np.zeros((Nx-1,Ny),dtype=np.float32)
deltay=np.zeros((Ny,Nx-1),dtype=np.float32)
for i in range(1,Nx+1):
    cx[i]=cx[i-1]+dx_1[0][i-1]
for i in range(1,Ny+1):
    cy[i]=cy[i-1]+dy_1[i-1][0]
#for ratio length make up 
for i in range(0,Nx+1):
    cx[i]=(cx[i]/cx[-1])*L
for i in range(0,Nx+1):
    cy[i]=(cy[i]/cy[-1])*H
#calculation of center point of cells 
for i in range(0,Ny):
    node_y[i]=(cy[i]+cy[i+1])/2 
for i in range(0,Nx):
    node_x[i]=(cx[i]+cx[i+1])/2
print(cy,node_y)
for i in range(0,Nx):
    deltax_1[i]=node_x[i+1]-node_x[i]
for i in range(0,Ny):
    deltay_1[i]=node_y[i+1]-node_y[i]
print(deltax_1)
for j in range(0,Nx-1):
    for i in range(0,Nx):
        deltax[j][i]=deltax_1[i]
for j in range(0,Ny):
    for i in range(0,Ny-1):
        deltay[j][i]=deltay_1[i]
deltay=np.transpose(deltay)
##final delta_x and delta_y in one D
delta_x=deltax.reshape(-1)
delta_y=deltay.reshape(-1)
print(delta_x)

[0.         0.08189874 0.17198735 0.27108483 0.38009205 0.5       ] [0.04094937 0.12694305 0.22153609 0.32558844 0.44004603 0.        ]
[ 0.17198735  0.18918609  0.2081047   0.22891517 -0.88009205]
[ 0.17198735  0.18918608  0.2081047   0.22891517 -0.880092    0.17198735
  0.18918608  0.2081047   0.22891517 -0.880092    0.17198735  0.18918608
  0.2081047   0.22891517 -0.880092    0.17198735  0.18918608  0.2081047
  0.22891517 -0.880092  ]


In [42]:
dx_1[0]

array([0.2    , 0.22   , 0.242  , 0.2662 , 0.29282], dtype=float32)

In [4]:
len(dx)

11